In [ ]:
import random
import tqdm

## Feature Extraction

how do we get the features we use in training?

By default we just use every word.

In [3]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [4]:

feature_weights = {}

## Data Reading

Read in the data from the training and evaluation  (or finally test) sets

In [5]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [ ]:
x_train, y_train = read_xy_data(r"path of your training data")
x_dev, y_dev = read_xy_data(r"path of your eval data ")

In [8]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code
so now let's setup the magic our classifier : 

basically it classifies the labels of our features:

Negative will classifiy it as -1

Neutral will classifiy it  as  0

Positive will classifiy it as  1

## How we run the classifier?

In [ ]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [18]:
import random
import tqdm
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:    
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 109605.37it/s]


## Evaluation Code

How we evaluate the classifier:

In [21]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [22]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{0: 389, 1: 909, -1: 912}


In [23]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.8105102996254682
Dev/test accuracy: 0.6321266968325792


## Visualize feature weights

We can inspect the weights that were learned for various features. Below we show the largest, smallest, and randomly selected feature weights. Inspecting them may give insight into the learned classifier.

In [24]:
import random

k = 25
topk_features = sorted(feature_weights.items(), key=lambda x: -x[1])[:k]
bottomk_features = sorted(feature_weights.items(), key=lambda x: x[1])[:k]
randomk_features = random.sample(list(feature_weights.items()), k)

print("Top-k")
for feature in topk_features:
    print(feature)

print("\nBottom-k")
for feature in bottomk_features:
    print(feature)

print("\nRandom k")
for feature in randomk_features:
    print(feature)

Top-k
('solid', 19.0)
('powerful', 17.0)
('half-bad', 17.0)
('prose', 17.0)
('fast', 16.0)
('inventive', 16.0)
('appealing', 15.0)
('refreshing', 15.0)
('remarkable', 14.0)
('treat', 14.0)
('eyes', 14.0)
('Maybe', 14.0)
('vividly', 14.0)
('rare', 13.0)
('manages', 13.0)
('sharp', 13.0)
('terrific', 13.0)
('ability', 13.0)
('includes', 13.0)
('memorable', 12.0)
('fun', 12.0)
('buffs', 12.0)
('charming', 12.0)
('assured', 12.0)
('follow', 12.0)

Bottom-k
('stupid', -21.0)
('none', -17.0)
('suffers', -16.0)
('worst', -15.0)
('failure', -15.0)
('thinks', -15.0)
('TV', -14.0)
('pretentious', -14.0)
('mess', -14.0)
('repetitive', -14.0)
('bland', -13.0)
('flat', -13.0)
('bore', -13.0)
('contrived', -13.0)
('depress', -13.0)
('harder', -13.0)
('Lawrence', -13.0)
('were', -12.0)
('depressing', -12.0)
('sticky', -12.0)
('alienating', -12.0)
('Unfortunately', -12.0)
('tired', -12.0)
('lacking', -12.0)
('lousy', -12.0)

Random k
('clouds', 2.0)
('Orc', 2.0)
('ick', -1.0)
('Black-and-white', -3.0)